In [1]:
# %load STMprotocol.py
import serial
import struct
import datetime
import time

class STMprotocol:
    def __init__(self, serial_port):
        self.ser = serial.Serial(serial_port, 115200, timeout=0.2)
        self.pack_format = {
            0x01: "=BBBB",
            0x03: "=Bf",
            0x04: "=B",
            0x05: "=B",
            0x06: "=Bffff",
            0x07: "=B",
            0x08: "=fff",
            0x09: "=",
            0x0a: "=",
            0x0b: "=BH",
            0x0c: "=B",
            0x0d: "=B",
            
            0xa0: "=fff",
            0xa1: "=fff",
        }

        self.unpack_format = {
            0x01: "=BBBB",
            0x03: "=BB",
            0x04: "=BB",
            0x05: "=BB",
            0x06: "=BB",
            0x07: "=ffff",
            0x08: "=BB",
            0x09: "=fff",
            0x0a: "=fff",
            0x0b: "=BB",
            0x0c: "=f",
            0x0d: "=BB",
            
            0xa0: "=Bfff",
            0xa1: "=BB",
        }

    def send_command(self, cmd, args):
        # Clear buffer
        print(self.ser.read(self.ser.in_waiting))
        
        parameters = bytearray(struct.pack(self.pack_format[cmd], *args))
        print(parameters)
        msg_len = len(parameters) + 5
        msg = bytearray([0xfa, 0xaf, msg_len, cmd]) + parameters
        crc = sum(msg) % 256
        msg += bytearray([crc])

        print("send ", repr(msg))
        self.ser.write(msg)

        start_time = datetime.datetime.now()
        time_threshold = datetime.timedelta(seconds=1)
        dt = start_time - start_time
        
        time.sleep(0.001)
        data = self.ser.read()[0]
        while (data != 0xfa) and (dt < time_threshold):
            data = self.ser.read()[0]

            current_time = datetime.datetime.now()
            dt = start_time - current_time

        adr = self.ser.read()[0]
        answer_len = self.ser.read()[0]
        answer = bytearray(self.ser.read(answer_len - 3))
        print("answer ", repr(bytearray([data, adr, answer_len]) + answer))

        args = struct.unpack(self.unpack_format[cmd], answer[1:-1])
        return args


In [2]:
protocol = STMprotocol("COM5")

In [3]:
# Echo
protocol.send_command(0x01, [ord(c) for c in 'ECHO'])

b''
bytearray(b'ECHO')
send  bytearray(b'\xfa\xaf\t\x01ECHO\xd2')
answer  bytearray(b'\xfa\xfa\t\x01ECHO\x1d')


(69, 67, 72, 79)

In [ ]:
# Set duty cycle of particular PWM channel
channel = 1
dutyCycle = 0.1
protocol.send_command(0x03, [channel, dutyCycle])

In [ ]:
# Set direction bit for particular motor
channel = 1
protocol.send_command(0x04, [channel])

In [ ]:
# Reset direction bit for particular motor
channel = 1
protocol.send_command(0x05, [channel]);

In [ ]:
# Set speed of all motors to 0

protocol.send_command(0x06, [4, 0, 0, 0, 0])

In [ ]:
# Set speed of all motors in rad/s

protocol.send_command(0x06, [4, 6, 6, 6, 6])

In [ ]:
# Read speed of all wheels in rad/s
numberOfmotors = 4
protocol.send_command(0x07, [numberOfmotors])

In [ ]:
# Set robot speed in its own coordinate system
Vx = 0.0
Vy = 0.0
Wz = 3.1415/2
protocol.send_command(0x08, [Vx, Vy, Wz])

In [ ]:
# Set robot speed in its own coordinate system
Vx = 0.0
Vy = 0.0
Wz = 0.0

protocol.send_command(0x08, [Vx, Vy, Wz])

In [ ]:
# Read robot speed in its own coordinate system
protocol.send_command(0x09, [])

In [ ]:
# Read robot coord in its own coordinate system
protocol.send_command(0x0a, [])

In [169]:
# Move servo of particular id (first parameter), to specified angle (second parameter) 
servoId = 2
angle = 255
protocol.send_command(0x0b, [servoId, angle])

b''
bytearray(b'\x02\xff\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x02\xff\x00\xbd')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')


(79, 75)

In [168]:
# Get angle of particular servo
servoId = 2
protocol.send_command(0x0c, [servoId])

b''
bytearray(b'\x02')
send  bytearray(b'\xfa\xaf\x06\x0c\x02\xbd')
answer  bytearray(b'\xfa\xfa\t\x0c\x00\x00\x00\x00\t')


(0.0,)

In [ ]:
# Turn on/off forward kinematics calculations
status = 1
protocol.send_command(0x0D, [status])

In [ ]:
# Get status
protocol.send_command(0xa0, [1, 1, 1])

In [81]:
# Parameters for right manipulator
rightSliderId = 1
rightGripperId = 16

rightSliderUpPos = 45
rightSliderBotPos = 300
rightGripperOpened = 40
rightGripperClosed = 150

In [158]:
# Open and lower manipulator
protocol.send_command(0x0b, [rightGripperId, rightGripperOpened])
time.sleep(0.5)
protocol.send_command(0x0b, [rightSliderId, rightSliderBotPos])

b''
bytearray(b'\x10(\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x10(\x00\xf4')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')
b''
bytearray(b'\x01,\x01')
send  bytearray(b'\xfa\xaf\x08\x0b\x01,\x01\xea')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')


(79, 75)

In [159]:
# Close and lift manipulator
protocol.send_command(0x0b, [rightGripperId, rightGripperClosed])
time.sleep(0.5)
protocol.send_command(0x0b, [rightSliderId, rightSliderUpPos])

b''
bytearray(b'\x10\x96\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x10\x96\x00b')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')
b''
bytearray(b'\x01-\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x01-\x00\xea')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')


(79, 75)

In [160]:
# Open and lower manipulator
protocol.send_command(0x0b, [rightGripperId, rightGripperOpened])
time.sleep(0.5)
protocol.send_command(0x0b, [rightSliderId, rightSliderBotPos])

b''
bytearray(b'\x10(\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x10(\x00\xf4')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')
b''
bytearray(b'\x01,\x01')
send  bytearray(b'\xfa\xaf\x08\x0b\x01,\x01\xea')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')


(79, 75)

In [161]:
# Close and lift manipulator
protocol.send_command(0x0b, [rightGripperId, rightGripperClosed])
time.sleep(0.5)
protocol.send_command(0x0b, [rightSliderId, rightSliderUpPos])

b''
bytearray(b'\x10\x96\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x10\x96\x00b')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')
b''
bytearray(b'\x01-\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x01-\x00\xea')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')


(79, 75)

In [162]:
# Open and lower manipulator
protocol.send_command(0x0b, [rightGripperId, rightGripperOpened])
time.sleep(0.5)
protocol.send_command(0x0b, [rightSliderId, rightSliderBotPos])

b''
bytearray(b'\x10(\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x10(\x00\xf4')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')
b''
bytearray(b'\x01,\x01')
send  bytearray(b'\xfa\xaf\x08\x0b\x01,\x01\xea')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')


(79, 75)

In [167]:
# Close and lift manipulator
protocol.send_command(0x0b, [rightGripperId, rightGripperClosed])
time.sleep(0.5)
protocol.send_command(0x0b, [rightSliderId, rightSliderUpPos])

b''
bytearray(b'\x10\x96\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x10\x96\x00b')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')
b''
bytearray(b'\x01-\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x01-\x00\xea')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')


(79, 75)

In [166]:
# Lower and open
protocol.send_command(0x0b, [rightSliderId, rightSliderBotPos])
time.sleep(1)
protocol.send_command(0x0b, [rightGripperId, rightGripperOpened])

b''
bytearray(b'\x01,\x01')
send  bytearray(b'\xfa\xaf\x08\x0b\x01,\x01\xea')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')
b''
bytearray(b'\x10(\x00')
send  bytearray(b'\xfa\xaf\x08\x0b\x10(\x00\xf4')
answer  bytearray(b'\xfa\xfa\x07\x0bOK\xa0')


(79, 75)

In [ ]:
Vx = 0.0
Vy = 0.0
Wz = 3.1415926/2
protocol.send_command(0x08, [Vx, Vy, Wz])

time.sleep(3.3)

Vx = 0.0
Vy = 0.0
Wz = 0.0
protocol.send_command(0x08, [Vx, Vy, Wz])

time.sleep(0.3)

Vx = 0.0
Vy = 0.1
Wz = 0.0
protocol.send_command(0x08, [Vx, Vy, Wz])

time.sleep(4)

Vx = 0.0
Vy = 0.0
Wz = 0.0
protocol.send_command(0x08, [Vx, Vy, Wz])

time.sleep(0.3)

Vx = 0.1
Vy = 0.0
Wz = 0.0
protocol.send_command(0x08, [Vx, Vy, Wz])

time.sleep(4)

Vx = 0.0
Vy = 0.0
Wz = 0.0
protocol.send_command(0x08, [Vx, Vy, Wz])

time.sleep(0.3)

Vx = -0.1
Vy = -0.1
Wz = 0.0
protocol.send_command(0x08, [Vx, Vy, Wz])

time.sleep(4)

Vx = 0.0
Vy = 0.0
Wz = 0.0
protocol.send_command(0x08, [Vx, Vy, Wz])